<a href="https://colab.research.google.com/github/Hing-9/popup_store/blob/main/KcElectra_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Graduation-Thesis/NER

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Graduation-Thesis/NER


In [ ]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModel

In [ ]:
data[0]

{'sn': 'ECOAR1R00432',
 'data_set': '일상생활및구어체',
 'domain': '해외고객과의채팅',
 'subdomain': '숙박,음식점',
 'en_original': 'Wow, that is a really generous offer, AAA!',
 'en': 'Wow, that is a really generous offer, AAA!',
 'mt': '와, 정말 후한 제안입니다, AAA!',
 'ko': '와, 정말 끌리는 제안이네요, AAA!',
 'source_language': 'en',
 'target_language': 'ko',
 'word_count_ko': 5.0,
 'word_count_en': 8.0,
 'word_ratio': 0.625,
 'file_name': '해외고객과의채팅_숙박,음식점.xlsx',
 'source': '크라우드 소싱',
 'license': 'open',
 'style': '구어체',
 'included_unknown_words': False,
 'ner': {'text': '와, 정말 끌리는 제안이네요, <PERSON>AAA</PERSON>!',
  'tags': [{'tag': 'PERSON', 'value': 'AAA', 'position': '[17, 20]'}]}}

In [ ]:
def file_to_data(path) :
    with open(path, 'r', encoding='UTF-8') as f :
        return json.load(f)['data']

def data_only_org(data) :
    ner_sentences = []
    for sentence in data :
        if sentence['ner'] : # NER 있는 데이터 발견
            tags = sentence['ner']['tags']
            org_flag = False
            for tag in tags :
                if tag['tag'] == 'ORG' :
                    org_flag = True
                    break
            if org_flag :
                ner_sentences.append(sentence)
    return ner_sentences

In [ ]:
data_영한 = file_to_data('./일상생활및구어체_영한_train_set.json')
data_한영 = file_to_data('./일상생활및구어체_한영_train_set.json')

In [ ]:
# 데이터 결합
data = data_영한
data.extend(data_한영)

assert len(data) != len(data_영한)+len(data_한영)

In [ ]:
print(f'전체 데이터 개수 : '+str(len(data)))
data = data_only_org(data)
print(f'NER에 ORG가 포함된 데이터 개수 : '+str(len(data)))

전체 데이터 개수 : 6000307
NER 데이터 개수 : 42853


In [ ]:
def kind_of_ner_tags(data) :
    result = []
    for sentence in data :
        tags = sentence['ner']['tags']
        for tag in tags :
            result.append(tag['tag'])
    
    result = list(set(result))
    return result

def value_for_tag(find_tag, data) :
    result = []
    MAX_CNT = 10
    cnt = 0

    for sentence in data :
        tags = sentence['ner']['tags']
        for tag in tags :
            if tag['tag'] == find_tag :
                result.append(tag['value'])
                cnt += 1
        
        if cnt >= MAX_CNT :
                break;
    
    return result

In [ ]:
tags = kind_of_ner_tags(data)
print(f'모든 데이터에서 검출된 NER TAGS : {tags}')
print()

for tag in tags :
    print(f'==== TAG : {tag}====')
    print(value_for_tag(tag, data))

모든 데이터에서 검출된 NER TAGS : ['ORG', 'WORK_OF_ART', 'PRODUCT', 'PERSON']

==== TAG : ORG====
['BBB1의', 'BBB2는', 'BBB', 'BBB2', 'BBB1 코리아', 'BBB1', 'BBB1은', 'BBB1', 'BBB', 'BBB 여행사']
==== TAG : WORK_OF_ART====
['FFF2', 'FFF쇼', 'FFF', 'FFF', 'FFF1', 'FFF2의', 'FFF', 'FFF', '어벤져스', '트리플A']
==== TAG : PRODUCT====
['IoT', '구글 어시스턴트', 'FFF 스마트 워치', 'FFF 립밤', 'FFF', 'FFF', 'FFF', '구워 먹어', 'AAA1', 'FFF 소프트웨어']
==== TAG : PERSON====
['AAA', 'AAA2입니다', 'AAA1', 'AAA', 'AAA', 'AAA2라고', 'AAA1입니다', 'AAA', 'AAA', 'AA1']


In [ ]:
# 혹시나.. 팝업이 들어간 데이터를 추출해보자
def find_popup(data) : 
    result = []
    for sentence in data :
        if '팝업' in sentence['ko'] :
            result.append(sentence['ko'])
        # elif '스토어' in sentence['ko'] :
        #     result.append(sentence['ko'])
    return result

find_popup(data)

['만석장롯데월드몰점 팝업에 갈래?',
 '행복한머릿꾼에서 진행하는 이번 팝업 이벤트는 정말 대박이에요.',
 '온라인 소매업체인 땡잡았네는 고객의 쇼핑 행동과 인구 통계를 이해하기 위해 팝업 스토어를 열었어요.',
 '미팅 플래너 및 기업 린시스템는 제품 및 브랜드 출시를 위한 팝업을 개최해요.',
 '글쎄, 대왕부동산은 팝업 수영장 파티를 조직하고 있어요.',
 '복지상담카페7142 스토어의 팝업 이벤트를 주최해 달라는 요청을 받았어요.',
 '난바우동에서 크리스마스에 팝업 행사를 개최한대.',
 '팝업 행사는 미르트레이딩 출시한 새로운 다과에 대한 프로모션 역할을 했습니다.',
 '새로운 고불초쌈밥 팝업 스토어 보셨어요?',
 '또보겠지상암DMC히어로점 팝업 이벤트에도 참여할 거야?',
 '또한 H키친김포금쌀떡갈비팝업행사치에이치헤어제품 인증마크를 받았습니다.',
 '또한 H키친김포금쌀떡갈비팝업행사치에이치헤어제품 인증마크를 받았습니다.',
 '또한 H키친김포금쌀떡갈비팝업행사치에이치헤어제품 인증마크를 받았습니다.',
 '또한 H키친김포금쌀떡갈비팝업행사치에이치헤어제품 인증마크를 받았습니다.']

In [ ]:
# ORG 태그에 브랜드명을 넣어 학습시키는게 정당해보임.
# 브랜드명은 소상공인 데이터
brand_data = pd.read_csv('./소상공인시장진흥공단_상가(상권)정보_서울_202212.csv', encoding='UTF-8')

In [ ]:
brand_names = brand_data['상호명'].values

In [ ]:
# 랜덤하게 브랜드명을 입력
import random

rnd_idx = random.randrange(0, len(brand_names))
brand_names[rnd_idx]

'장미화원'

In [ ]:
def random_brand_transfer(raw_data) :
    data = raw_data.copy()
    flag=False
    for sentence in data :
        if not sentence['ner'] :
            continue
        
        tags = sentence['ner']['tags']
        for tag in tags :
            if tag['tag'] == 'ORG' :

                rnd_idx = random.randrange(0, len(brand_names))
                chg_org = brand_names[rnd_idx]

                positions = tag['position'][1:-1].split(',')
                start_index = int(positions[0])
                end_index = int(positions[1])
                
                target = sentence['ko']
                target = target[:start_index] + chg_org + target[end_index:]
                sentence['ko'] = target

                # 일단 전체문장만 변경하였으나, 추후 tag의 position 등의 전처리도 필요할듯!

    return data

chg_data = random_brand_transfer(data)

In [ ]:
chg_data[3]

{'sn': 'ECOAR3T00922',
 'data_set': '일상생활및구어체',
 'domain': '해외고객과의채팅',
 'subdomain': '숙박,음식점',
 'en_original': 'BBB2 is a Korean fried chicken restaurant or locally called chimaek restaurant.',
 'en': 'BBB2 is a Korean fried chicken restaurant or locally called chimaek restaurant.',
 'mt': 'BBB2는 한국식 프라이드 치킨 레스토랑 또는 현지에서 치맥 레스토랑이라고 합니다.',
 'ko': '치맥라이프 한국의 프라이드 치킨 전문점 또는 치맥 전문점입니다.',
 'source_language': 'en',
 'target_language': 'ko',
 'word_count_ko': 8.0,
 'word_count_en': 12.0,
 'word_ratio': 0.667,
 'file_name': '해외고객과의채팅_숙박,음식점.xlsx',
 'source': '크라우드 소싱',
 'license': 'open',
 'style': '구어체',
 'included_unknown_words': False,
 'ner': {'text': '<ORG>BBB2는</ORG> 한국의 프라이드 치킨 전문점 또는 치맥 전문점입니다.',
  'tags': [{'tag': 'ORG', 'value': 'BBB2는', 'position': '[0, 5]'}]}}